In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
# A sample torch model with layer attribute
# inps: Number of input features
# hiddens: Number of neurons on each layer, 
#     e.g., [] means no hidden layer, 
#     [128] means one hidden layer with 128 neurons
# bias: Decide if there is a bias on each layer, must be true in the example
# seed: Reproductivity, None means random seed, otherwise specifiy a integer
# hidden_activation: Activation function after each hidden layer

class TorchNNCore(torch.nn.Module):
    def __init__(
        self, inps, hiddens=[], bias=True, seed=None, hidden_activation=torch.nn.ReLU
    ):
        super(TorchNNCore, self).__init__()
        if seed is not None:
            torch.manual_seed(seed)
        struct = [inps] + hiddens + [1]
        self.layers = [] # This layer attribute is required under 
        for i in range(1, len(struct)):
            self.layers.append(
                torch.nn.Linear(
                    in_features=struct[i - 1], out_features=struct[i], bias=bias
                )
            )
            if i == len(struct) - 1:
                self.layers.append(torch.nn.Sigmoid())
            else:
                self.layers.append(hidden_activation())
        self.model = torch.nn.Sequential(*self.layers)

    def forward(self, x):
        output = self.model(x)
        return output

In [3]:
# Prepare training & testing dataset
data = pd.read_csv('./adult.csv').to_numpy()
X_train = torch.tensor(data[:,:-1], dtype=torch.float)
y_train = torch.tensor(data[:,-1].reshape(-1,1), dtype=torch.float)
print(X_train.shape, y_train.shape)

torch.Size([45222, 98]) torch.Size([45222, 1])


In [4]:
# Specify loss function, define model and optimizer
loss_func = torch.nn.BCELoss()
model = TorchNNCore(inps=X_train.shape[1], hiddens=[128], hidden_activation=torch.nn.LeakyReLU)
optim = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
y_train_np = y_train.detach().numpy()
for epoch in range(0,300):
    optim.zero_grad()
    y_pred = model(X_train)
    loss = loss_func(y_pred, y_train)
    loss.backward()
    optim.step()
    if epoch%10==0:
        y_pred_np = (y_pred.detach().numpy()) > 0.5
        accuracy = sum(y_pred_np == y_train_np)/y_train_np.shape[0]
        print('Epoch = %d, loss = %.4f, accuracy=%.4f'%(epoch, loss.tolist(), accuracy))
optim.zero_grad()

Epoch = 0, loss = 3.0384, accuracy=0.7749


In [ ]:
# Before using influence function, we show the structure of the model
print(model)

In [7]:
# And we print the "layer" attribute, which is used to fetch the layers above
for item in model.layers:
    print(item)

Linear(in_features=98, out_features=128, bias=True)
LeakyReLU(negative_slope=0.01)
Linear(in_features=128, out_features=1, bias=True)
Sigmoid()


In [8]:
# Define InfluenceFunction class
# model: A input pytorch model, must be trained and have the "layer" attribute
#        If you got "LinAlgError: Singular matrix" error, try to change the activation functions
#        of the model, e.g., change from torch.nn.ReLU to torch.nn.LeakyReLU
# X_train: Feature matrix used to train model
# y_train: Feature matrix used to train model
# loss_func: Pre-defined loss function for the trained model
# layer_index: The layer whose parameters are used for the calculation, usually the last linear layer

from InfluenceFunction import InfluenceFunction

infl = InfluenceFunction(
    model = model, # Warning: the class will take a snapshot of the model, any further change requires new instance
    X_train = X_train, # Features, must be torch.Tensor
    y_train = y_train, # Labels, must be torch.Tensor
    loss_func = loss_func, # In this example, it's BCELoss
    layer_index = -2, # In this example, as shown in the model structure, we use the second last layer 
)

In [9]:
# Example of influence on removing records
for index in range(0,10):
    print(infl.influence_remove_single(index))

# Note: The influence scores depend the current status of the model,
# even for the same model configuration with different random seed,
# the model may converge to different point in hyperspace, which will
# result in different score for the same records.
# Note2: The absolute value of the influece score is meaningless, but
# they are comparable with each other.

0.2292262995561718
0.1218828616273201
3.448065179385834
-0.0
-0.15204049717484447
-0.0
0.4896758370652158
-0.19838219001583357
-0.0
-0.19294061795491238


In [10]:
# Adding an attribute with given x and y
# Here as an arbitrary example we fetch No.24 record and add its duplicate
index = 24
x = X_train[index].reshape(1,-1).detach()
y = y_train[index].reshape(1,-1).detach()
print(infl.influence_add_single(x,y))

0.5389216042751486


In [11]:
# Modifying x and y, e.g., flipping y to 1-y
# Here as an arbitrary example we flip the label of No.48 record
index = 48
x = X_train[index].reshape(1,-1).detach()
y = y_train[index].reshape(1,-1).detach()
print(infl.influence_modify_single(index,x,1-y))

-4.148492037639535
